In [ ]:
import os
os.sys.path.append('../src/')

%load_ext autoreload
%autoreload 2

In [ ]:
import gentrification as gn
import gentriviz as gv
import pandas as pd
import geopandas as gpd
from tqdm import tqdm

import hvplot.pandas
import holoviews as hv
from holoviews.operation.datashader import datashade, spread
from holoviews.element import tiles
from holoviews import opts

import os
from dotenv import load_dotenv

load_dotenv()
PATH = os.environ.get('PATH_RAW')

BG_TILES = tiles.CartoLight()
hv.extension('bokeh')

In [ ]:
# Define some helper functions

def filter_corporaties(gdf: gpd.GeoDataFrame, 
                       corporaties: gpd.GeoDataFrame):
    if gdf.crs != 28992:
        gdf = gdf.to_crs(28992)
    if corporaties.crs != 28992:
        corporaties = corporaties.to_crs(28992)
    gdf = gdf.sjoin(corporaties[['categorie', 'geometry']], predicate='intersects', how='left')
    #gdf = gdf[gdf.naam.isna()].copy()
    gdf.drop(['categorie', 'index_right'], axis=1, inplace=True)
    
    return gdf

def write_scenarios(df, scenario, jaar):
    return df.to_csv(PATH + f'for_viz/{jaar}_{scenario}.csv', index=False)

## Import data

In [ ]:
# Import WOZ-waarden with geometries
woz = gn.import_data('woz_bag')

# Import neighborhood data (CBS takes a while)

cbs = gn.import_data('cbs')
ses_woa = gn.import_data('ses_woa')
leefbaarometer = gn.import_data('leefbaarometer')
overwaarde = gn.import_data('overwaarde')
gemeenten = gn.import_data('gemeenten')
zh = gpd.read_parquet(PATH + 'filters/zuid_holland_corporaties.parquet')
nh = gpd.read_file(PATH + 'filters/amsterdam_metropool_corporaties.geojson')
corp = gpd.read_parquet(PATH + 'filters/corporaties_vves.parquet')
cbswijk_2016 = gpd.read_file(PATH + 'cbs/wijkbuurt2016/wijk_2016.shp')
cbsbuurt_2016 = gpd.read_file(PATH + 'cbs/wijkbuurt2016/buurt_2016.shp')
cbswijk_2022 = gpd.read_file(PATH + 'cbs/wijkbuurt2022/wijk_2022_v1.shp')
cbsbuurt_2022 = gpd.read_file(PATH + 'cbs/wijkbuurt2022/buurt_2022_v1.shp')

print(f'''Imported\n--------\nBAG: {bag.shape[0]} rows and {bag.shape[1]} columns
WOZ: {woz.shape[0]} rows and {woz.shape[1]} columns
CBS: {cbs.shape[0]} rows and {cbs.shape[1]} columns
SES_WOA: {ses_woa.shape[0]} rows and {ses_woa.shape[1]} columns
LEEFBAAROMETER: {leefbaarometer.shape[0]} rows and {leefbaarometer.shape[1]} columns
OVERWAARDE: {overwaarde.shape[0]} rows and {overwaarde.shape[1]} columns
WOONPLAATSEN: {gemeenten.shape[0]} rows and {gemeenten.shape[1]} columns''')

## Create corporatiefile

We kunnen de eigenaarsinfo van 2019 gebruiken (RVO-file), maar daar zitten ook percelen in van VVE's. Nu is het verschil vaak niet heel groot, maar uit eerdere analyse blijkt dat deze file het aantal beschikbare objecten onderschat (hangt van de stad af). Het verschil voor Rotterdam is vrij klein, voor Zoetermeer vrij groot. Het is mogelijk om het corporatiebezit in de RVO file te vervangen voor een meer betrouwbare files van Zuid-Holland, de Metropoolregio Amsterdam en Nijmegen. Deze file noemen we corp_strict.


In [ ]:
# get provincie

provincies = gpd.read_file(PATH + 'cbs/cbsgebiedsindelingen2023.gpkg', layer='provincie_gegeneraliseerd').to_crs(28992)

# Select provincies
prov = ['Utrecht']

to_remove = provincies[provincies.statnaam.isin(prov)]

# Select MRA

mra = gpd.read_file(PATH + 'filters/mra.geojson').to_crs(28992)

# Select Nijmegen

gemeente = ['Nijmegen']
gemeenten_to_remove = gemeenten[gemeenten.gemeente.isin(gemeente)]

nijmegen = gpd.read_file(PATH + 'filters/nijmegen_corporaties.geojson').to_crs(28992)

# Remove provinces and MRA

corp_strict = corp.loc[~corp.intersects(to_remove.unary_union)].reset_index(drop=True)
corp_strict = corp_strict.loc[~corp_strict.intersects(mra.unary_union)].reset_index(drop=True)
corp_strict = corp_strict.loc[~corp_strict.intersects(gemeenten_to_remove.unary_union)].reset_index(drop=True)

# Rename and clean columns

provincies.drop(['statcode', 'jrstatcode', 'rubriek', 'id'], axis=1, inplace=True)
provincies = provincies.rename(columns={'statnaam': 'categorie'})
corp = corp.rename(columns={'categorie_pbl': 'categorie'})


nijmegen.drop(['id', 'DD_BIJGEWERKT', 'ID', 'ZAKRECHT', 'NAAM'], axis=1, inplace=True)
nijmegen = nijmegen.rename(columns={'NAAM_UPPER': 'categorie'})
mra = mra.rename(columns={'Gemeentenaam': 'categorie'})
mra.drop(['Gemeente_ID', 'Gemeentecode', 'Provincie'], axis=1, inplace=True)
nh = nh[['geometry', 'Gemeente']].copy()
nh = nh.rename(columns={'Gemeente': 'categorie'})
zh = zh[['wl_woonplaatsnaam', 'geometry']].copy()
zh = zh.rename(columns={'wl_woonplaatsnaam': 'categorie'})


corp_strict = pd.concat([corp_strict, zh.to_crs(28992), nh.to_crs(28992), nijmegen])
corp_strict = corp_strict[['geometry', 'categorie']].copy()

## Create maps

In [ ]:
params = {'min_oppervlakte': 80,
          'max_oppervlakte': 200,
          'plaats': ['Rotterdam'],
          'income1': 38500,
          'income2': 25500,
          'region': False,
          'overwaarde': False,
          'mortgage_range': 0.9,
          'divorced': False}

In [ ]:
# Get local dataset

dict_2016, dict_2022 = gn.compare_jaren(gdf=woz, **params)
df16 = dict_2016.get('df')
df22 = dict_2022.get('df')
print(f'2016: {len(df16)} objects\n2022: {len(df22)} objects')

In [ ]:
df16_filtered = filter_corporaties(df16, corp_strict)
df22_filtered = filter_corporaties(df22, corp_strict)
print(f'2016: {len(df16_filtered)} objects\n2022: {len(df22_filtered)} objects')

In [ ]:
onv = gv.plot_histograms(df16_filtered, df22_filtered, ses_woa, leefbaarometer, 'onv')
woz_dif = gv.plot_histograms(df16_filtered, df22_filtered, ses_woa, leefbaarometer, 'woz')
lbm = gv.plot_histograms(df16_filtered, df22_filtered, ses_woa, leefbaarometer, 'lbm')
verm = gv.plot_histograms(df16_filtered, df22_filtered, ses_woa, leefbaarometer, 'vermogen_percentielgroep')
soc = gv.plot_histograms(df16_filtered, df22_filtered, ses_woa, leefbaarometer, 'soc')
afw = gv.plot_histograms(df16_filtered, df22_filtered, ses_woa, leefbaarometer, 'ses_woa_score')
maps = gv.plot_maps(df16_filtered, df22_filtered, **params)

(maps + onv + woz_dif + lbm + verm + soc + afw).cols(2)

In [ ]:
# Write filtered data to file for visualisation


scenario = 'amsterdam_region_35000_overwaarde_sochuisgefilterd_90m2'

dfs = [df16_filtered, df22_filtered]
jaren = ['2016', '2022']
for df, jaar in zip(dfs, jaren):
    df = df.to_crs(4326)
    df['latitude'] = df.geometry.y
    df['longitude'] = df.geometry.x
    df = df[['woz_2016_m2', 'woz_2022_m2', 'woz_difference_abs', 'woz_difference_postcode_5', 'latitude', 'longitude']].copy()
    write_scenarios(df, scenario, jaar)

## Create stats 2016-2022

Stappen:
1. Importeer CBS buurten en wijken
2. Kies profiel
3. Bereken aanbod en verschillende metrics

In [ ]:
# Get buurten geometries

buurten = gpd.read_file(PATH + 'cbs/wijkbuurt2023/buurt_2023_v0.shp')
buurten = buurten[buurten['WATER'] == 'NEE'][['geometry', 'BU_CODE', 'BU_NAAM', 'GM_NAAM']].copy()
buurten.rename(columns={'BU_CODE': 'buurtcode', 'BU_NAAM': 'buurtnaam', 'GM_NAAM': 'gemeente'}, inplace=True)

wijken = gpd.read_file(PATH + 'cbs/wijkbuurt2023/wijk_2023_v0.shp')
wijken = wijken[wijken['WATER'] == 'NEE'][['geometry', 'WK_CODE', 'WK_NAAM', 'GM_NAAM']].copy()
wijken.rename(columns={'WK_NAAM': 'wijknaam', 'WK_CODE': 'wijkcode', 'GM_NAAM': 'gemeentenaam'}, inplace=True)

In [ ]:
# Add stedelijkheid

sted = pd.read_csv(PATH + 'cbs/stedelijkheid.csv', sep=';', skiprows=1, names=['naam', 'gemeente', 'regio_soort', 'regiocode', 'stedelijkheid'])
sted.regiocode = sted.regiocode.str.strip()

wijken = pd.merge(wijken,
                  sted[['regiocode', 'stedelijkheid']],
                  left_on='wijkcode',
                  right_on='regiocode',
                  how='left')
wijken.drop('regiocode', axis=1, inplace=True)

buurten = pd.merge(buurten,
                   sted[['regiocode', 'stedelijkheid']],
                   left_on='buurtcode',
                   right_on='regiocode',
                   how='left')
buurten.drop('regiocode', axis=1, inplace=True)

buurten.stedelijkheid = buurten.stedelijkheid.astype('Int64')
wijken.stedelijkheid = wijken.stedelijkheid.astype('Int64')

In [ ]:
gemeentelist = gemeenten.gemeente.to_list()
len(gemeentelist)

In [ ]:
# Create profile

params = {'min_oppervlakte': 80,
          'max_oppervlakte': 200,
          'income1': 38500,
          'income2': 25500,
          'region': False,
          'overwaarde': False,
          'mortgage_range': 0.85,
          'divorced': False}

In [ ]:
overwaardes = [{'overwaarde': True}, {'overwaarde': False}]
tags = ['overwaarde', 'geen_overwaarde']

dfs = []

for wp in gemeentelist:
    dfs_ = []
    for overwaarde, tag in zip(overwaardes, tags):
        params.update({'plaats': [wp]})
        params.update(overwaarde)
                    
        df16, df22 = gn.compare_jaren(woz, **params)

        # Add buurtparameters
        
        df16 = df16.get('df').sjoin(ses_woa[ses_woa.jaar==2016], predicate='intersects', how='left')
        df16.drop('index_right', axis=1, inplace=True)
        df16 = df16.sjoin(leefbaarometer, predicate='intersects', how='left')
        df16.drop('index_right', axis=1, inplace=True)

        df22 = df22.get('df').sjoin(ses_woa[ses_woa.jaar==2021], predicate='intersects', how='left')
        df22.drop('index_right', axis=1, inplace=True)
        df22 = df22.sjoin(leefbaarometer, predicate='intersects', how='left')
        df22.drop('index_right', axis=1, inplace=True)
        
        #groupby cols
        cols = ['gemeente']
        df16_ = df16.groupby(cols).agg(woz_waarde = ('woz_2016', 'median'),
                                       number_of_houses = len(df16),
                                       woz_difference = ('woz_difference_abs', 'median'),
                                       woz_difference_pc5 = ('woz_difference_postcode_5', 'median'),
                                       ses_woa = ('ses_woa_score', 'median'),
                                       lbm = ('lbm', 'median'),  
                                       fys = ('fys', 'median'),
                                       onv = ('onv', 'median'),
                                       soc = ('soc', 'median'),
                                       financiele_welvaart_percentielgroep = ('financiele_welvaart_percentielgroep', 'median'),
                                       vermogen_percentielgroep = ('vermogen_percentielgroep', 'median')
                                        ).reset_index()
        
        df16_['number_of_houses'] = len(df16)
        
        df16 = pd.melt(df16_,
                    id_vars=cols,
                    value_vars=['ses_woa', 'lbm', 'woz_difference', 'number_of_houses'],
                    var_name='metric',
                    value_name=f'2016_{tag}')
        
        df22_ = df22.groupby(cols).agg(woz_waarde = ('woz_2022', 'median'),
                                       number_of_houses = len(df22),
                                       woz_difference = ('woz_difference_abs', 'median'),
                                       woz_difference_pc5 = ('woz_difference_postcode_5', 'median'),
                                       ses_woa = ('ses_woa_score', 'median'),
                                       lbm = ('lbm', 'median'),  
                                       fys = ('fys', 'median'),
                                       onv = ('onv', 'median'),
                                       soc = ('soc', 'median'),
                                       financiele_welvaart_percentielgroep = ('financiele_welvaart_percentielgroep', 'median'),
                                       vermogen_percentielgroep = ('vermogen_percentielgroep', 'median')
                                    ).reset_index()
        
        df22_['number_of_houses'] = len(df22)
        df22 = pd.melt(df22_,
                       id_vars=cols,
                       value_vars=['ses_woa', 'lbm', 'woz_difference', 'number_of_houses'],
                       var_name='metric',
                       value_name=f'2022_{tag}')
        
        df = pd.merge(df16, 
                      df22, 
                      on=['gemeente', 'metric'],
                      how='left')
        
        dfs_.append(df)
    
    df = pd.merge(dfs_[0],
                  dfs_[1],
                  on=['gemeente', 'metric'],
                  how='left')
        
        
    dfs.append(df)

df = pd.concat(dfs).reset_index(drop=True)

names = {'number_of_houses': 'Aantal huizen',
         'woz_difference': 'Groei waarde 2016-2022',
         'woz_difference_pc5': 'Groei waarde 2016-2022 PC5',
         'ses_woa': 'Sociaal-economisch',
         'lbm': 'Leefbaarheid',
         'fys': 'Fysieke leefomgeving',
         'onv': 'Onveiligheid', 
         'soc': 'Sociale cohesie',
         'financiele_welvaart_percentielgroep': 'Financiële welvaart',
         'vermogen_percentielgroep': 'Vermogen'
         }
        
df.replace({'metric': names}, inplace=True)


len(df)

In [ ]:
df_ = pd.merge(df, gemeenten, left_on='gemeente', right_on='gemeente', how='left')
df_ = gpd.GeoDataFrame(df_, geometry='geometry', crs=28992)
df_ = df_.to_crs(3857)
df_.to_file(PATH + 'for_viz/gemeenten.geojson')

In [ ]:

df.to_csv(PATH + 'for_viz/gemeenten_for_viz.csv', index=False)